<a href="https://colab.research.google.com/github/AraiKensuke/LOST/blob/Upgrade_python3/FromLocalFile.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:

#########  clone and install necessary packages from github repositories

import sys
import importlib

if importlib.util.find_spec("pyPG") is None:
  !git clone https://github.com/AraiKensuke/pyPG.git
  %cd /content/pyPG/
   ###  pyPG is written in C++/python, and uses the C library GNU Scientific Library.  
  !sudo apt-get install libgsl-dev    
  !python setup.py build_ext --inplace

if importlib.util.find_spec("LOST") is None:
  !git clone https://github.com/AraiKensuke/LOST.git
  %cd /content/LOST

  !python setup.py build_ext --inplace

sys.path.insert(1, "/content/LOST")    #  add these to searchpath for python modules
sys.path.insert(1, "/content/pyPG")    #  add these to searchpath for python modules


#from google.colab import files
#uploaded = files.upload()


In [ ]:
"""
Template to invoke Latent Oscillation in Spike Train (LOST) model

There are 
"""

import numpy as _N
import os 
from tmrsclTest import timeRescaleTest, zoom
from mcmcFigs import getComponents, plotWFandSpks, showFsHist, filterByFsAmps, plotPSTH, corrcoeffs, plotFsAmp, acorrs
from ARcfSmplFuncs import dcmpcff
import mcmcARp_h_osc as _mARp
#import mcmcARp as _mARp
import inspect
import __main__ as main
import scipy.stats as _ss
import mcmcFigs as mF
import pickle
import parseFN as pF
import utilities as _U

#_plt.ioff()

###################  params and settings
bRealDat         = False      #  is our data real data <lfp> <fltrd> <spk 0 1>
pkldSmpls        = "smplsN.dump"   #  last Gibbs samples, pickled for continuation
postModes        = None       #  MAP estimate of parameters
dontrun          = False      #  just construct mARp from pickle

__exec_base_fn__ = inspect.getfile(inspect.currentframe())[0:-3]
nwpr, tr0, tr1, CnpCs, R = pF.parseFN(__exec_base_fn__)
Cn               = CnpCs - 1

if not os.access(__exec_base_fn__, os.F_OK):
    os.mkdir(__exec_base_fn__)

mARp = _mARp.mcmcARp()
if nwpr == "wp":
    mARp.sig_ph0H =-(0.97*0.97)

mARp.mcmcRunDir= __exec_base_fn__
mARp.burn     = 1500
mARp.NMC      = 0
mARp.peek     = 200
mARp.model    = "bernoulli"
mARp.t0       = 0
mARp.t1       = 1200
mARp.ID_q2    = False
mARp.bpsth    = True
mARp.dt       = 0.001
mARp.histFN   = ""
mARp.ifs      = [(0.3 / mARp.fSigMax) * _N.pi]    #  initial values
mARp.Cn       = Cn
mARp.R        = R
mARp.knownSigFN= None
mARp.xknownSig=0.9
mARp.BsmpxSkp = 10
mARp.doBsmpx  = True
mARp.noAR     = False
mARp.processes= 1
latentITERS   = 200   #  when doing runLatent



if (pkldSmpls is not None) and (postModes is not None):
    print "set either pckldSmpls (continue Gibbs) or postModes (use to generate latent state, once Gibbs hits stationarity), but not both"
    exit

"""
No more parameters to fiddle with below this line.  
Runs the actual Gibbs sampler based on above params
"""
pckl = None
bRunLatent = False
if (pkldSmpls is not None) or (postModes is not None):
    bRunLatent = False if (pkldSmpls is not None) else True

    if pkldSmpls == "smplsN.dump":  #  allow for restart Gibbs
        ##  [in = None, out = smpls-1.dump]   [in = smpls-1.dump, out = smpls-2.dump]
        lfn, ufn = _U.uniqFN("%s/smpls.dump" % __exec_base_fn__, serial=True, returnPrevious=True)
        mARp.outSmplFN = ufn.split("/")[1]
        pickleFN = lfn.split("/")[1] if lfn is not None else None

    else:
        pickleFN   = postModes

    if pickleFN is not None:  #  None if first time running
        dmp = open("%(1)s/%(2)s" % {"1" : __exec_base_fn__, "2" : pickleFN}, "rb")
        pckl = pickle.load(dmp)
        dmp.close()

if not bRunLatent:   #  regular run.  
    mARp.run(runDir=__exec_base_fn__, trials=_N.arange(tr0, tr1), pckl=pckl, runlatent=False, dontrun=dontrun)
    
    if not dontrun:
        plotFsAmp(mARp, dir=__exec_base_fn__)
        mARp.dump_smps(dir=__exec_base_fn__)
else:     #  generate latents
    mARp.burn = latentITERS
    mARp.NMC  = 0
    mARp.run(runDir=__exec_base_fn__, trials=_N.arange(tr0, tr1), pckl=pckl, runlatent=True, dontrun=dontrun)
    if not dontrun:
        mARp.dump(__exec_base_fn__)

        oscMn = _N.mean(mARp.Bsmpx[:, (latentITERS/2):, 2:], axis=1)
        mdn  = mF.plotWFandSpks(mARp, mARp.Bsmpx[:, :, 2:], norm=False, cls2EvryO=range(20, latentITERS), bRealDat=bRealDat, sFilename="%s/latnt,GrdTr,Spks" % __exec_base_fn__)

        _N.savetxt("%s/oscMn.dat" % __exec_base_fn__, oscMn.T, fmt=("% .4f " * mARp.TR))


